In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [2]:
links = ['https://www.rappi.com.co/restaurantes/900068701',
         'https://www.rappi.com.co/restaurantes/900017229-tigre-comida-china',
         'https://www.rappi.com.co/restaurantes/900086547-kfc-pollo',
         'https://www.rappi.com.co/restaurantes/900012595-papa-johns']
products = []
descriptions = []
prices = []
images = []
names = []
style = []
restaurant = []

In [3]:
for link in links:
    url = requests.get(link)
    soup = BeautifulSoup(url.content,'html.parser')
    info = soup.find_all('script')[3].text
    data = json.loads(info)

    all_items = data['hasMenu']['hasMenuSection']
    for i in range(len(all_items)):
        sections = all_items[i][0]['hasMenuItem']
        for j in range(len(sections)):
            products.append(sections[j][0]['name'])
            descriptions.append(sections[j][0]['description'])
            prices.append(sections[j][0]['offers']['price'])
            restaurant.append(data['name'])

    for tag in soup.find_all('img',attrs={'class':'sc-1d781697-1'}):
        images.append(tag['src'])
        names.append(tag['alt'])
        style.append(tag['style'])

In [4]:
data_images = pd.DataFrame([],columns=[])
data_images['imagen'] = images
data_images['productos'] = names
data_images['style'] = style
data_images = data_images[data_images['imagen'].str.contains('images.rappi')]
data_images = data_images[~data_images['style'].str.contains('image:url')]

data_rappi = pd.DataFrame([], columns=[])
data_rappi['productos'] = products
data_rappi['descripcion'] = descriptions
data_rappi['precio'] = prices
data_rappi['restaurante'] = restaurant
data_rappi['imagen'] = data_rappi.set_index('productos').index.map(data_images.set_index('productos')['imagen'])

In [5]:
data_rappi.to_csv("./data_rappi.csv", index=False,  encoding = 'latin1')